# Python 2 HSUTCC: Session 7: Special Task

## Special Task (Bonus 8% - Deadline Thursday 27 Nov 2025)

Symbolic differentiation allows to calculate a value of a derivative for an expression in a form of a sequence of symbols. Humans are likely to make mistakes during a long monotonous work and calculating of a direvative even not that hard but tidious process. Systems of symbolic calculations, like in this <a href='https://www.wolframalpha.com/input?i=derivate+x%5E3+++(1+-+x)+/+(1+-+x%5E2)'>example</a> are dedicated to save humanity from the need to spend their time for a potentially wrong calcuations.

We will try to implement a simple system of symbolic differentiation on Python. The main type in our system will be a class `Expr` – an expression. All successor of the class should implement:

- `__call__` method which calculates a value of an expression in a specified context: the context bounds names of variables in the expression with specific values.
- `d` (Leibniz's notation of derivative) method which takes a name of a variable wrt (from “with respect to”) and returns an expression for a derivative with respect to this variable.

In [ ]:
class Expr:
    """Base class for all symbolic expressions."""
    def __call__(self, **context):
        """Calculates the value of the expression given a context (variable bindings)."""
        raise NotImplementedError

    def d(self, wrt):
        """Returns the symbolic derivative of the expression with respect to the variable 'wrt'."""
        raise NotImplementedError
    
    def __str__(self):
        """Returns the S-expression representation of the formula."""
        raise NotImplementedError

# --- 1. Constant ---
class Const(Expr):
    """Represents a constant numeric value."""
    def __init__(self, value):
        self.value = value
        
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        # Derivative of a constant is 0
        return Const(0)
    
    def __str__(self):
        return str(self.value)

# --- 2. Variable ---
class Var(Expr):
    """Represents a variable (e.g., 'x', 'y')."""
    def __init__(self, name):
        self.name = name
        
    def __call__(self, **context):
        # Retrieve the variable's value from the context
        if self.name not in context:
            raise ValueError(f"Variable '{self.name}' not defined in context.")
        return context[self.name]
    
    def d(self, wrt):
        # Derivative of x wrt x is 1; derivative of y wrt x is 0
        if self.name == wrt:
            return Const(1)
        else:
            return Const(0)
            
    def __str__(self):
        return self.name

# --- 3. Addition ---
class Add(Expr):
    """Represents the sum of two expressions (u + v)."""
    def __init__(self, u, v):
        self.u = u
        self.v = v
        
    def __call__(self, **context):
        return self.u(**context) + self.v(**context)
    
    def d(self, wrt):
        # Sum rule: d/dx(u + v) = d/dx(u) + d/dx(v)
        return Add(self.u.d(wrt), self.v.d(wrt))
        
    def __str__(self):
        # S-expression format: (+ u v)
        return f"(+ {self.u} {self.v})"

# --- 4. Multiplication ---
class Mul(Expr):
    """Represents the product of two expressions (u * v)."""
    def __init__(self, u, v):
        self.u = u
        self.v = v
        
    def __call__(self, **context):
        return self.u(**context) * self.v(**context)
    
    def d(self, wrt):
        # Product rule: d/dx(u * v) = d/dx(u) * v + u * d/dx(v)
        term1 = Mul(self.u.d(wrt), self.v)
        term2 = Mul(self.u, self.v.d(wrt))
        return Add(term1, term2)
        
    def __str__(self):
        # S-expression format: (* u v)
        return f"(* {self.u} {self.v})"

**Important**: For each class, implement a `__str__` method which should return a corresponding formula in a form of S-expression.

1. Implement class for two types of expressions: `Const` – a constant and `Var` – a variable. For convinience next we will use not a constructors of classes but their one-letter synonyms:

```python
V = Var
C = Const
```

Example of class usage:

---

```python
C(42)() # print(C(42)) -> 42
```

```terminal
42
```

---

```python
C(42).d(V("x"))() # print(C(42).d(V("x"))) -> 0
```

```terminal
0
```

---

```python
V("x")(x=42) # print(V("x")) -> x
```

```terminal
42
```

---

```python
V("x").d(V("x"))() # print(V("x").d(V("x"))) -> 1
```

```terminal
1
```

---

```python
V("x").d(V("y"))() # print(V("x").d(V("y"))) -> 0
```

---

In [ ]:
# Your code here
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)
    
    def __str__(self) -> str:
        return str(self.value)

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):

        if isinstance(wrt, Var):
            wrt = wrt.name

        if self.name == wrt:
            return Const(1)
        return Const(0)
    
    def __str__(self):
        return self.name



In [ ]:
V = Var
C = Const

print('C(42)() ->',C(42)())
print('C(42).d(V(\'x\'))() ->', C(42).d(V('x'))())
print('V(\'x\') ->', V('x'))
print('V(\'x\')(x = 42) ->', V('x')(x = 42))
print('V(\'x\').d(V(\'x\'))() ->', V('x').d(V('x'))())
print('V(\'x\').d(V(\'y\'))() ->', V('x').d(V('y'))())

C(42)() -> 42
C(42).d(V('x'))() -> 0
V('x') -> x
V('x')(x = 42) -> 42
V('x').d(V('x'))() -> 1
V('x').d(V('y'))() -> 0


2. Implement classes for binary operations: `Sum`, `Product`, and `Fraction`. Binary operations by definition work with exactly two operands that’s why a constructor for all binary classes will be the same. It’s convinient to take it out into a separate base class:


In [ ]:
class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

class Neg(Expr):
    def __init__(self, expr):
        self.expr = expr
    
    def __call__(self, **context):
        return -self.expr(**context)
    
    def d(self, wrt):
        return Neg(self.expr.d(wrt))
    
    def __str__(self):
        return f'(- {self.expr})'

An example of using of some binary operations:

---

```python
# x + x^2
x = V("x")
Sum(x, Product(x, x)).d(x)(x=42) # (+ 1 (+ (* 1 x) (* x 1)))
```

```terminal
85
```

---

```python
Product(x, Sum(x, C(2)))(x=42) # (* x (+ x 2))
```

```terminal
1848
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)
```

```terminal
0.14285714285714285
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)
```

```terminal
0.5
```

---

In [ ]:
# Your code here
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x) + g(x)]/dx = f'(x) + g'(x)
        '''
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self):
        return f'(+ {self.expr1} {self.expr2})'

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)*g(x)]/dx = f(x)*g'(x) + f'(x)*g(x)
        '''
        return Sum(
            Product(self.expr1, self.expr2.d(wrt)),
            Product(self.expr1.d(wrt), self.expr2)
            )

    def __str__(self):
        return f'(* {self.expr1} {self.expr2})'

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)/g(x)]/dx = [f'(x)g(x) - f(x)g'(x)]/[g(x)*g(x)]
        '''
        return Fraction(
            Sum(
                Product(self.expr1.d(wrt), self.expr2),
                Neg(Product(self.expr1, self.expr2.d(wrt)))
            ),
            Product(self.expr2, self.expr2)
        )
    
    def __str__(self):
        return f'(/ {self.expr1} {self.expr2})'

In [ ]:
x = V("x")
print(Sum(x, Product(x, x)).d(x))
Sum(x, Product(x, x)).d(x)(x=42)

(+ 1 (+ (* x 1) (* 1 x)))


85

In [ ]:
print(Product(x, Sum(x, C(2))))
Product(x, Sum(x, C(2)))(x=42)

(* x (+ x 2))


1848

In [ ]:
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)

0.14285714285714285

In [ ]:
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)

0.5

3. Implement a class `Power` for an operation of an exponentiation operation. For simplicity you can expect that a power – is an expression consisting only of constants.

---

```python
Power(Fraction(V("x"), C(4)), C(2))(x=42) # (** (/ x 4) 2)
```

```terminal
110.25
```

---

```python
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)
```

```terminal
5.25
```

In [ ]:
class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x)^n] = (n*f(x)^(n-1))*f'(x)
        '''
        return Product(
            Product(self.expr2, # n * (f(x) ^ (n-1))
                    Power(self.expr1, # f(x) ^ (n-1)
                          Sum(self.expr2, Neg(Const(1))))), # n-1
            self.expr1.d(wrt)
        )
    
    def __str__(self):
        return f'(** {self.expr1} {self.expr2})'

In [ ]:
print(Power(Fraction(V("x"), C(4)), C(2)))
Power(Fraction(V("x"), C(4)), C(2))(x=42)

(** (/ x 4) 2)


110.25

In [ ]:
# f(x) = (x/4)^2 = (x^2)/16
# f'(x) = x/8
# f'(42) = 42/8 = 5.25
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)

5.25

4. Implement an overloading of arithmetic operators for a base class Expr. Overall there are needed to be implement 7 operators: 2 unary and 5 binary:

```terminal
-e          e.__neg__()
+e          e.__pos__()
e1 + e2     e1.__add__(e2)
e1 - e2     e1.__sub__(e2)
e1 * e2     e1.__mul__(e2)
e1 / e2     e1.__truediv__(e2)
e1 ** e2    e1.__pow__(e2)
```

---

```python
((C(1) - V("x")) ** C(3) + V("x"))(x=12)
```

```terminal
-1319
```

---

In [ ]:
class Expr:
    def __call__(self, **context):
        pass
    
    def d(self, wrt):
        pass

    # Overloading of arithmetic operators
    def __neg__(e):
        return Neg(e)

    def __pos__(e):
        return e
    
    def __add__(e1, e2):
        return Sum(e1, e2)

    def __sub__(e1, e2):
        return Sum(e1, Neg(e2))

    def __mul__(e1, e2):
        return Product(e1, e2)

    def __truediv__(e1, e2):
        return Fraction(e1, e2)
    
    def __pow__(e1, e2):
        return Power(e1, e2)

In [ ]:
# To test your implementation, put all other classes here
# Restart the kernel, run the above and current cell, then do some testing
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)
    
    def __str__(self) -> str:
        return str(self.value)

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):

        if isinstance(wrt, Var):
            wrt = wrt.name

        if self.name == wrt:
            return Const(1)
        return Const(0)
    
    def __str__(self):
        return self.name

C = Const
V = Var

class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

class Neg(Expr):
    def __init__(self, expr):
        self.expr = expr
    
    def __call__(self, **context):
        return -self.expr(**context)
    
    def d(self, wrt):
        return Neg(self.expr.d(wrt))
    
    def __str__(self):
        return f'(- {self.expr})'

class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x) + g(x)]/dx = f'(x) + g'(x)
        '''
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self):
        return f'(+ {self.expr1} {self.expr2})'

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)*g(x)]/dx = f(x)*g'(x) + f'(x)*g(x)
        '''
        return Sum(
            Product(self.expr1, self.expr2.d(wrt)),
            Product(self.expr1.d(wrt), self.expr2)
            )

    def __str__(self):
        return f'(* {self.expr1} {self.expr2})'

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)/g(x)]/dx = [f'(x)g(x) - f(x)g'(x)]/[g(x)*g(x)]
        '''
        return Fraction(
            Sum(
                Product(self.expr1.d(wrt), self.expr2),
                Neg(Product(self.expr1, self.expr2.d(wrt)))
            ),
            Product(self.expr2, self.expr2)
        )
    
    def __str__(self):
        return f'(/ {self.expr1} {self.expr2})'

class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x)^n] = (n*f(x)^(n-1))*f'(x)
        '''
        return Product(
            Product(self.expr2, # n * (f(x) ^ (n-1))
                    Power(self.expr1, # f(x) ^ (n-1)
                          Sum(self.expr2, Neg(Const(1))))), # n-1
            self.expr1.d(wrt)
        )
    
    def __str__(self):
        return f'(** {self.expr1} {self.expr2})'

# Your code here
print(((C(1) - V("x")) ** C(3) + V("x"))(x=12))
((C(1) + V('x')) ** C(2)).d('x')(x=3) # 2x + 2

-1319


8